### Import the text representation from local module

In [99]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

### Import the necessary dependencies to extract the text from the pdf

In [100]:
import numpy as np
import pdfparser.poppler as pdf
from simple_text_representation.classes.Text import Text
from simple_text_representation.classes.Paragraph import Paragraph
from simple_text_representation.classes.Sentence import Sentence
from simple_text_representation.classes.Word import Word
from dateutil.parser import parse

### Set global variables that describe the structure of the pdf file

In [101]:
INITIAL_PAGE = 5
PAGES_PER_TEXT = 2
ALLOWED_TAGS = ['APRENDEMOS', 'PRACTICAMOS', 'EXTENSIÓN']

In [102]:
def convertPDFtoText(fileName):
    document = pdf.Document(fileName.encode('UTF-8'))

    print('No of pages', document.no_of_pages)
#     for page in document:
#         print('Page', page.page_no, 'size =', page.size)
#         for f in page:
# #             print(' '*1,'Flow')
#             for b in f:
# #                 print(' '*2,'Block', 'bbox=', b.bbox.as_tuple())
#                 for l in b:
#                     print(' '*3, l.text)
#                     #assert l.char_fonts.comp_ratio < 1.0
# #                     for i in range(len(l.text)):
# #                         print(l.text[i].encode('UTF-8'), '(%0.2f, %0.2f, %0.2f, %0.2f)'% l.char_bboxes[i].as_tuple(),\
# #                             l.char_fonts[i].name, l.char_fonts[i].size, l.char_fonts[i].color,)
#                     print()
#                 print("=============================================================================")
    return document

In [103]:
document = convertPDFtoText("../files/comprension_lectora_2.pdf")

No of pages 194


# Limpieza de Textos

## Estructura actual de un Texto

Despues de realizar la estracción de textos con la herramienta PDFtoText se consigue un listado de elementos, donde cada elemento representa una parte de un texto y cada par de elementos adyacentes representan un texto completo. En su estado original, estos traen mucha información innecesaria que debe ser removida y que detallará a continuación.

* Tema a tratar
* Etiqueta de la sección
* Indicaciones del texto
* Titulo del texto
* Referencias de imagenes
* Preguntas relacionas al texto.
* Referencias del texto
* Pie de pagina
* Descripción de la hoja actual.


## Retos de la Limpieza

* Conservar la estructura original del texto, es decir, que todos los parrafos sean identificables.
* Eliminar todas las referencias de imagenes existentes, ya que se usan de forma indeterminada en distintas partes de un texto.
* Adaptar las diferentes estructuras de los archivos PDF provistos

## Pasos de Limpieza

* Usar la Etiqueta de cada texto para determinar el inicio de este.
    * Remover la información previa a la etiqueta
    * Remover la información antes del inicio del texto y despues de la etiqueta
* Eliminar la información extra por hoja.
    * Eliminar la información extra de la primera pagina (referencias y pie de pagina)
    * Eliminar la información extra de la segunda pagina (referencias, preguntas y pie de pagina)
* Eliminar las referencias a imagenes aún existentes.

## Resultado esperado

Finalmente, se espera obtener un arreglo de elementos donde cada represente a un texto completo y que este listo para ser pre-procesado.

In [104]:
def hasAllowedTag(line):
    return any(TAG in line for TAG in ALLOWED_TAGS)

In [105]:
def transformToPython(page):
    listOfFlows = list()
    for flow in page:
        listOfBlocks = list()
        for paragraph in flow:
            listOfLines = ''
            for line in paragraph:
                listOfLines = joinLine(listOfLines, line.text)
#                 listOfLines.append(line.text)
#                 print(line.text)
#                 print('====================================================================================')
            listOfBlocks.append(listOfLines)
#             listOfBlocks.append(''.join(listOfLines))
        listOfFlows.append(listOfBlocks)

    return listOfFlows

In [106]:
def joinLine(listOfLines, line):
    newListOfLines = listOfLines
    separator = ''
    
    if (newListOfLines):
        if (newListOfLines[- 1] == '-'):
            newListOfLines = newListOfLines[:-1]
        else:
            separator = ' '
        newListOfLines = newListOfLines + separator + line
    else:
        newListOfLines = newListOfLines + line

    return newListOfLines

In [107]:
def cleanDocument(file):
    nextPageAdded = -1
    listOfTexts = list()
    text = list()
    
    for index,page in enumerate(file):
        if (index > INITIAL_PAGE):
            if (nextPageAdded == 1):
                text.append(transformToPython(page))
                listOfTexts.append(text)
                nextPageAdded = -1
            else:
                text = list()
                listOfFlows = list()
                for flow in page:
                    listOfBlocks = list()
                    for paragraph in flow:
                        listOfLines = ''
                        for line in paragraph:
                            listOfLines = joinLine(listOfLines, line.text)
#                             listOfLines.append(line.text)
                            if (hasAllowedTag(line.text)):
                                nextPageAdded = 1
#                             print(line.text)
#                         listOfBlocks.append(''.join(listOfLines))
                        listOfBlocks.append(listOfLines)
#                         print("================================================================")
                    listOfFlows.append(listOfBlocks)
                text.append(listOfFlows)

    return listOfTexts

### List of unformated Texts

In [108]:
listOfTexts = cleanDocument(document)

### Clean conditions

In [109]:
def hasLinkReference(block):
    return 'http' in block

In [110]:
def onlyOneCharacter(block):
    return len(block) == 1

In [111]:
def isOnlyDate(block):
    try: 
        parse(block)
        return True
    except ValueError:
        return False

In [112]:
def specialCases(block):
    return ('Libro Comunicacion' in block) or ('Comprensión lectora' in block)

In [114]:
def isStartOfQuestionary(block):
    return 'Responde las preguntas' in block

In [115]:
def hasIndications(block):
    return 'Lee el siguiente text' in block

In [113]:
def cleanContiditions(block):
    return not (hasLinkReference(block) or onlyOneCharacter(block) or isOnlyDate(block) or specialCases(block))

In [128]:
def cleanPage(page, isFirstPage = False):
    newPage = list()
    isTagFound = False
    questionary = False
    isIndications = False

    for flow in page:
        listOfBlocks = list()
        for block in flow:
            if (cleanContiditions(block)):
                listOfBlocks.append(block)
            isTagFound = isTagFound or hasAllowedTag(block)
            isIndications = isIndications or hasIndications(block)
            questionary = questionary or isStartOfQuestionary(block)

        if (questionary):
            break;
        if isTagFound or isIndications:
            newPage = list()
            listOfBlocks = list()
            isTagFound = False
            isIndications = False
        if (len(listOfBlocks) > 0):
            newPage.append(listOfBlocks)

    print(newPage)
    print('========================================================================')
    if (newPage and (isFirstPage and len(newPage) > 1)):
        title = newPage[0][0]
        newPage = newPage[1:][0]
        return title, newPage
    elif (newPage and not isFirstPage):
        return newPage[0]
    else:
        return '', []

### After declaring all the methods needed to clean the text, we do a simple text with the first two pages

In [129]:
testFirstPage = listOfTexts[0][0]
testSecondPage = listOfTexts[0][1]

In [130]:
testFirstPage

[['2.o grado de secundaria',
  'Ficha: Identificamos la estructura de los textos instructivos',
  'APRENDEMOS',
  'Lee el siguiente texto considerando las orientaciones que brinda tu docente.'],
 ['El lavado de las manos',
  'El lavado de manos con jabón es una de las maneras más efectivas y económicas de prevenir enfermedades diarreicas y respiratorias (como la gripe, la neumonía, la bronquitis, etc.). Además, con este procedimiento, pueden evitarse las parasitosis (infecciones por parásitos) y algunas infecciones en la piel y los ojos (como la conjuntivitis y el orzuelo). Si nuestras manos están sucias y nos tocamos los ojos, la nariz o la boca, podemos contagiarnos de enfermedades tan frecuentes como la gripe.'],
 ['¿Cómo lavarse las manos?'],
 ['Humedecer las manos con agua.'],
 ['Aplicar suficiente jabón para cubrir la superficie de ambas manos.'],
 ['Frotar las palmas de las manos entre sí.'],
 ['Frotar la palma derecha sobre el dorso de la izquierda entrelazando los dedos, y vic

In [131]:
cleanPage(testFirstPage, True)

[['El lavado de las manos', 'El lavado de manos con jabón es una de las maneras más efectivas y económicas de prevenir enfermedades diarreicas y respiratorias (como la gripe, la neumonía, la bronquitis, etc.). Además, con este procedimiento, pueden evitarse las parasitosis (infecciones por parásitos) y algunas infecciones en la piel y los ojos (como la conjuntivitis y el orzuelo). Si nuestras manos están sucias y nos tocamos los ojos, la nariz o la boca, podemos contagiarnos de enfermedades tan frecuentes como la gripe.'], ['¿Cómo lavarse las manos?'], ['Humedecer las manos con agua.'], ['Aplicar suficiente jabón para cubrir la superficie de ambas manos.'], ['Frotar las palmas de las manos entre sí.'], ['Frotar la palma derecha sobre el dorso de la izquierda entrelazando los dedos, y viceversa.'], ['Frotar palma contra palma, entrelazando los dedos.'], ['Frotar el dorso de los dedos contra la palma de la mano opuesta.'], ['Frotar con un movimiento de rotación el pulgar izquierdo atrapá

('El lavado de las manos', ['¿Cómo lavarse las manos?'])

In [132]:
testSecondPage

[['Comprensión lectora 2'],
 ['¿Cuándo debes lavarte las manos?'],
 ['Datos interesantes'],
 ['• Antes y después de manipular alimentos o amamantar. • Antes de comer o beber y después de manipular basura o desperdicios. • Después de tocar alimentos crudos y antes de tocar alimentos cocidos. • Después de ir al baño y luego de cambiarle los pañales al bebé. • Después de sonarse la nariz, toser o estornudar. • Luego de haber tocado objetos como dinero, llaves, pasamanos, etc. • Cuando se llega a la casa de la calle, del trabajo y de la escuela. • Antes y después de curar heridas o de atender a alguien que está enfermo.'],
 ['• Debes lavarte las manos con frecuencia entre 30 y 50 segundos. • Se estima que 1 de cada 3 personas no se lava las manos después de ir al baño. • Lavarse las manos cuesta, más o menos, entre 70 a 90 céntimos, pero permite ahorrar mucho dinero en medicamentos y consultas médicas.'],
 ['Utiliza el texto anterior para responder las siguientes preguntas:', '1'],
 ['¿Cuá

In [133]:
cleanPage(testSecondPage)

[['¿Cuándo debes lavarte las manos?'], ['Datos interesantes'], ['• Antes y después de manipular alimentos o amamantar. • Antes de comer o beber y después de manipular basura o desperdicios. • Después de tocar alimentos crudos y antes de tocar alimentos cocidos. • Después de ir al baño y luego de cambiarle los pañales al bebé. • Después de sonarse la nariz, toser o estornudar. • Luego de haber tocado objetos como dinero, llaves, pasamanos, etc. • Cuando se llega a la casa de la calle, del trabajo y de la escuela. • Antes y después de curar heridas o de atender a alguien que está enfermo.'], ['• Debes lavarte las manos con frecuencia entre 30 y 50 segundos. • Se estima que 1 de cada 3 personas no se lava las manos después de ir al baño. • Lavarse las manos cuesta, más o menos, entre 70 a 90 céntimos, pero permite ahorrar mucho dinero en medicamentos y consultas médicas.'], ['Utiliza el texto anterior para responder las siguientes preguntas:'], ['¿Cuánto debe durar el lavado de manos?', '

['¿Cuándo debes lavarte las manos?']

In [97]:
def transformDocument(listOfUTexts):
    listOfText = list()

    for firstPage, secondPage in listOfUTexts:
        arrParagrahps = list()
        print('~~~~~~~~~~~')
        print(firstPage)
        print('---')
        print(secondPage)
        text = Text()
        title, arrParagraphs = cleanPage(firstPage, True)
        if (arrParagraphs):
            arrParagrahps.append(cleanPage(secondPage))
            text.setTitle(title)
            text.setUnformatedParagraphs(arrParagrahps)
            text.formatParagraphs()
            listOfTexts.append(text)

    return listOfTexts

In [98]:
result = transformDocument(listOfTexts)
result 

~~~~~~~~~~~
[['1.er grado de secundaria'], ['Ficha: Reconocemos la estructura del texto argumentativo', 'APRENDEMOS'], ['Lee el siguiente texto considerando las orientaciones que brinda tu docente.'], ['El acto solidario de la donación de órganos1'], ['Fuente de imagen: <https://goo.gl/exLu3M>'], ['Si bien los trasplantes se han convertido en una práctica habitual, aún persisten fuertes temores en la población para donar órganos, lograr su superación es la clave para aumentar el número de los donadores solidarios que hacen falta para salvar miles de vidas.', 'Es preciso, entonces, que se aclaren algunas dudas para que las personas pierdan el miedo a donar. Primero, que lo complicado de los procedimientos de extirpación y trasplantación, en el que intervienen varios equipos médicos altamente especializados, vuelve muy difícil la existencia de mafias. Segundo, que la necesaria compatibilidad (afinidad de grupo sanguíneo) entre donante y receptor dificulta la posibilidad de muertes “a ped

TypeError: 'Text' object is not iterable